In [0]:
from pyspark.sql import functions as F

silver_path = "s3://capstone-impacta/Capstone/silver/posts"
gold_path   = "s3://capstone-impacta/Capstone/gold/video_views_dia_faculdade"


In [0]:
df_silver = (
    spark.read
        .format("delta")
        .load(silver_path)
)


In [0]:
df_base = (
    df_silver
        .withColumn("data_postagem", F.to_date("data_postagem"))
)


In [0]:
df_gold_video_views_dia_faculdade = (
    df_base
        .groupBy("data_postagem", "faculdade")
        .agg(
            F.sum("views").alias("total_views"),
            F.sum("likes").alias("total_likes"),
            F.sum("comentarios").alias("total_comentarios"),
            F.count("*").alias("total_videos")
        ).orderBy("data_postagem")
)


In [0]:
(
    df_gold_video_views_dia_faculdade
        .coalesce(1)
        .write
        .mode("overwrite")
        .option("header", "true")
        .csv(gold_path)
)


In [0]:
#df_gold_video_views_dia_faculdade.display()